### Linear Regression Project - Predicting House Prices

Information about the dataset is available [here](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627)

Information about the columns is available [here](https://s3.amazonaws.com/dq-content/307/data_description.txt)

Data is available to download here [here](https://dsserver-prod-resources-1.s3.amazonaws.com/235/AmesHousing.txt)

#### Imports 

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("AmesHousing.txt", sep="\t")

In [3]:
df.head(3)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [6]:
def train_and_test(df, features):
    X = df[features].drop('SalePrice', axis=1)
    y = df['SalePrice']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    predictions = lr.predict(X_test)
    mse = mean_squared_error(predictions, y_test)
    return np.sqrt(mse)

In [7]:
def get_numeric_features(df):
    return df.select_dtypes(include=np.number).columns.to_list()

In [8]:
def select_features():
    return ['Gr Liv Area', 'SalePrice']

In [9]:
living_area_feature = select_features()
train_and_test(df, living_area_feature)

57585.82288777018

Using a linear regression model using only the above ground living area the RMSE is $57586

### Feature engineering 

* Any columns with more than 25% of missing values will be dropped
* Columns that leak data will be dropeed
* Data will be formatted including converting data to be of type category, scaling numerical data and imputing missing values
* Create new categories through combining original features

In [10]:
five_percent_of_df = len(df)//20
five_percent_of_df

146

In [11]:
null_counts = df.isnull().sum().sort_values(ascending=False)
null_counts.head(10)

Pool QC          2917
Misc Feature     2824
Alley            2732
Fence            2358
Fireplace Qu     1422
Lot Frontage      490
Garage Qual       159
Garage Yr Blt     159
Garage Cond       159
Garage Finish     159
dtype: int64

In [12]:
mv_features_to_drop = null_counts[null_counts > five_percent_of_df].index.to_list()
mv_features_to_drop

['Pool QC',
 'Misc Feature',
 'Alley',
 'Fence',
 'Fireplace Qu',
 'Lot Frontage',
 'Garage Qual',
 'Garage Yr Blt',
 'Garage Cond',
 'Garage Finish',
 'Garage Type']

In [14]:
num_missing = df.select_dtypes(include=np.number).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values().index.to_list()
fixable_numeric_cols

['BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Garage Cars',
 'Garage Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Mas Vnr Area']

In [15]:
df_fixable_modes = df[fixable_numeric_cols].mode().loc[0]
df_fixable_modes

BsmtFin SF 1      0.0
BsmtFin SF 2      0.0
Bsmt Unf SF       0.0
Total Bsmt SF     0.0
Garage Cars       2.0
Garage Area       0.0
Bsmt Full Bath    0.0
Bsmt Half Bath    0.0
Mas Vnr Area      0.0
Name: 0, dtype: float64

Function to drop cols with more than 5% of vals missing 
and if feature is numeric and has less than 5% missing than fill the missing value with the mode

In [16]:
def transform_features(df):
    df.drop(mv_features_to_drop, axis = 1, inplace=True)
    df.fillna(df_fixable_modes, inplace=True)

    return df

In [17]:
df_cleaned = transform_features(df)

In [18]:
df_cleaned.isnull().sum().sort_values(ascending=False)

Bsmt Exposure     83
BsmtFin Type 2    81
BsmtFin Type 1    80
Bsmt Cond         80
Bsmt Qual         80
                  ..
Central Air        0
Heating QC         0
Heating            0
Total Bsmt SF      0
Order              0
Length: 71, dtype: int64

In [19]:
df_cleaned[fixable_numeric_cols].isnull().sum()

BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Garage Cars       0
Garage Area       0
Bsmt Full Bath    0
Bsmt Half Bath    0
Mas Vnr Area      0
dtype: int64

In [20]:
df_cleaned.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
 

Drop columns that have data leakage - these are columns that you would not know when trying to predict the price of a house

In [21]:
df_cleaned = df_cleaned.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Drop columns that do not contain useful information 

In [22]:
df_cleaned = df_cleaned.drop(['Order', 'PID'], axis = 1)

In [23]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MS SubClass      2930 non-null   int64  
 1   MS Zoning        2930 non-null   object 
 2   Lot Area         2930 non-null   int64  
 3   Street           2930 non-null   object 
 4   Lot Shape        2930 non-null   object 
 5   Land Contour     2930 non-null   object 
 6   Utilities        2930 non-null   object 
 7   Lot Config       2930 non-null   object 
 8   Land Slope       2930 non-null   object 
 9   Neighborhood     2930 non-null   object 
 10  Condition 1      2930 non-null   object 
 11  Condition 2      2930 non-null   object 
 12  Bldg Type        2930 non-null   object 
 13  House Style      2930 non-null   object 
 14  Overall Qual     2930 non-null   int64  
 15  Overall Cond     2930 non-null   int64  
 16  Year Built       2930 non-null   int64  
 17  Year Remod/Add